In [17]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Data Prep

In [18]:
bc = datasets.load_breast_cancer()
# print(bc.DESCR)
X, y = bc.data, bc.target

n_samples, n_features = X.shape
print(n_samples, n_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

#scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))


# reshape y
# we reshape y because we want to have a 2D tensor with 1 column
# instead of a 1D tensor. This is because PyTorch expects a 2D tensor
# when we are dealing with a dataset with multiple features.
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

569 30


Model

In [19]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticRegression(n_features)

Loss and Optimizer

In [20]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

Training Loop

In [21]:
num_epochs = 100
for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)

    # backward pass
    loss.backward()

    # updates
    optimizer.step()

    # zero gradients
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

# Evaluate the model    
with torch.no_grad():
    y_predicted = model(X_test) # get the predictions
    y_predicted_cls = y_predicted.round() # round off the values to either 0 or 1
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0]) # the number of correct predictions divided by the number of samples
    print(f'accuracy: {acc}') 

epoch: 10, loss = 0.4668
epoch: 20, loss = 0.4134
epoch: 30, loss = 0.3737
epoch: 40, loss = 0.3428
epoch: 50, loss = 0.3180
epoch: 60, loss = 0.2976
epoch: 70, loss = 0.2804
epoch: 80, loss = 0.2657
epoch: 90, loss = 0.2529
epoch: 100, loss = 0.2417
accuracy: 0.9210526347160339
